# 1) Importing the data

In [1]:
# import packages
import pandas as pd
import seaborn as sns
import numpy as np
from datetime import date

In [3]:
# import data
dim_company_type = pd.read_csv(r'C:\Users\Dante\Desktop\MPG\CSV files\dim_company_type.csv')
dim_educational_qualification = pd.read_csv(r'C:\Users\Dante\Desktop\MPG\CSV files\dim_educational_qualification.csv')
dim_gender = pd.read_csv(r'C:\Users\Dante\Desktop\MPG\CSV files\dim_gender.csv')
dim_industry = pd.read_csv(r'C:\Users\Dante\Desktop\MPG\CSV files\dim_industry.csv')
dim_job_specialisation = pd.read_csv(r'C:\Users\Dante\Desktop\MPG\CSV files\dim_job_specialisation.csv')
dim_nationality = pd.read_csv(r'C:\Users\Dante\Desktop\MPG\CSV files\dim_nationality.csv')
dim_race = pd.read_csv(r'C:\Users\Dante\Desktop\MPG\CSV files\dim_race.csv')
dim_source = pd.read_csv(r'C:\Users\Dante\Desktop\MPG\CSV files\dim_source.csv')
dim_state = pd.read_csv(r'C:\Users\Dante\Desktop\MPG\CSV files\dim_state.csv')
dim_working_arrangement = pd.read_csv(r'C:\Users\Dante\Desktop\MPG\CSV files\dim_working_arrangement.csv')
fact_job_details = pd.read_csv(r'C:\Users\Dante\Desktop\MPG\CSV files\fact_job_details.csv')
fact_submissions = pd.read_csv(r'C:\Users\Dante\Desktop\MPG\CSV files\fact_submissions.csv')

## Merge fact_submissions

In [ ]:
fact_submissions.head()

In [ ]:
# join educational_qualification
fact_submissions = pd.merge(left = fact_submissions,
                            right = dim_educational_qualification,
                            how = 'left',
                           on = 'educational_qualification_key')
fact_submissions.drop(columns = ['educational_qualification_key', 'educational_qualification_bm'], inplace = True)

# join gender
fact_submissions = pd.merge(left = fact_submissions,
                            right = dim_gender,
                            how = 'left',
                            on = 'gender_key')
fact_submissions.drop(columns = ['gender_key', 'gender_bm'], inplace = True)

# join nationality
fact_submissions = pd.merge(left = fact_submissions,
                            right = dim_nationality,
                            how = 'left',
                            on = 'nationality_key')
fact_submissions.drop(columns = ['nationality_key', 'nationality_bm', 'code_alpha2', 'code_alpha3', 'region'], inplace = True)

# join race
fact_submissions = pd.merge(left = fact_submissions,
                            right = dim_race,
                            how = 'left',
                            on = 'race_key')
fact_submissions.drop(columns = ['race_key', 'race_bm'], inplace = True)

In [ ]:
fact_submissions.head()

## Merge fact_job_details

In [ ]:
fact_job_details.head()

In [ ]:
# join industry
fact_job_details = pd.merge(left = fact_job_details,
                            right = dim_industry,
                            how = 'left',
                           on = 'industry_key')
fact_job_details.drop(columns = ['industry_key', 'industry_bm', 'industry_sub_definitions'], inplace = True)

# join job_specialisation
fact_job_details = pd.merge(left = fact_job_details,
                            right = dim_job_specialisation,
                            how = 'left',
                           on = 'job_specialisation_key')
fact_job_details.drop(columns = ['job_specialisation_key', 'job_specialisation_bm', 'job_specialisation_sub_definitions'], inplace = True)

# join company_type
fact_job_details = pd.merge(left = fact_job_details,
                            right = dim_company_type,
                            how = 'left',
                            on = 'company_type_key')
fact_job_details.drop(columns = ['company_type_key', 'company_type_bm'], inplace = True)

# join working_arrangement
fact_job_details = pd.merge(left = fact_job_details,
                            right = dim_working_arrangement,
                            how = 'left',
                            on = 'working_arrangement_key')
fact_job_details.drop(columns = ['working_arrangement_key', 'working_arrangement_bm'], inplace = True)

# join state
fact_job_details = pd.merge(left = fact_job_details,
                            right = dim_state,
                            how = 'left',
                            on = 'state_key')
fact_job_details.drop(columns = ['state_key', 'state_bm'], inplace = True)

In [ ]:
fact_job_details.head()

# 2) Analysis

## Benford's Law

In [ ]:
# get first, second number of salary amounts
df_salary_int = pd.DataFrame()
df_salary_int['salary'] = fact_job_details['gross_salary_myr'].astype({'gross_salary_myr': 'int64'})
df_salary_int['first_digit'] = df_salary_int['salary'].astype(str).str[:1]
df_salary_int['second_digit'] = df_salary_int['salary'].astype(str).str[1:2]
df_salary_int

In [ ]:
# histogram for first digit
sns.countplot(data = df_salary_int,
              y = 'first_digit',
             order = ['1','2','3','4','5','6','7','8','9','0'])

In [ ]:
df_first_digit_counts = pd.DataFrame()
df_first_digit_counts = df_salary_int.groupby(['first_digit']).size().to_frame('count')
df_first_digit_counts['percentage'] = 100 * (df_first_digit_counts['count']/len(df_salary_int))
df_first_digit_counts

In [ ]:
# histogram for second digit
sns.countplot(data = df_salary_int,
              y = 'second_digit',
             order = ['1','2','3','4','5','6','7','8','9','0'])

In [ ]:
df_second_digit_counts = pd.DataFrame()
df_second_digit_counts = df_salary_int.groupby(['second_digit']).size().to_frame('count')
df_second_digit_counts['percentage'] = 100 * (df_second_digit_counts['count']/len(df_salary_int))
df_second_digit_counts

## Submission demographics

In [ ]:
fact_submissions.info()

In [ ]:
# make datetime type
fact_submissions = fact_submissions.astype({'created_at': 'datetime64'})

# separate date and time
fact_submissions['date'] = fact_submissions['created_at'].dt.date
fact_submissions['time'] = fact_submissions['created_at'].dt.time
fact_submissions = fact_submissions.astype({'date': 'datetime64'})

In [ ]:
fact_submissions.info()

In [ ]:
fact_submissions['date'].head()

In [ ]:
fact_submissions['time'].head()

In [ ]:
# generate series from 1st of launch month to today
min_date = np.min(fact_submissions['date'])
dates = pd.date_range(start = min_date, end = date.today(), freq='D')
df_dates = pd.Series(dates).to_frame()
df_dates.columns = ['date']

In [ ]:
df_dates.head()

In [ ]:
fact_submissions.info()

In [ ]:
# get submission count for each day
df_submissions_per_day = fact_submissions.groupby('date').size().to_frame('count')
df_submissions_per_day.reset_index(inplace = True)

In [ ]:
df_submissions_per_day = df_submissions_per_day.astype({'date': 'datetime64'})

In [ ]:
# add col for cumulative submission count
df_submissions_per_day['cumulative_count'] = df_submissions_per_day['count'].cumsum()

In [ ]:
df_submissions_per_day.info()

In [ ]:
# merge
df_submissions_per_day = pd.merge(left = df_dates,
                                  right = df_submissions_per_day,
                                  how = 'left',
                                  on = 'date')

In [ ]:
# add days since launch column
df_submissions_per_day['days_since_launch'] = df_submissions_per_day['date'] - min_date

# change to int
df_submissions_per_day['days_since_launch'] = df_submissions_per_day['days_since_launch'].dt.days

In [ ]:
# fill down
df_submissions_per_day['cumulative_count'].ffill(axis = 0, inplace = True)

# fillna
df_submissions_per_day['cumulative_count'].fillna(value = 0, inplace = True)

In [ ]:
df_submissions_per_day

In [ ]:
df_submissions_per_day.info()

In [ ]:
# timeline of submissions
sns.lineplot(data = df_submissions_per_day,
             x = 'date',
             y = 'cumulative_count',
             palette = 'pastel')

In [ ]:
# timeline of submissions days since launch
sns.lineplot(data = df_submissions_per_day,
             x = 'days_since_launch',
             y = 'cumulative_count',
             palette = 'pastel')

In [ ]:
# gender breakdown
sns.countplot(data = fact_submissions,
              x = 'gender_en',
              palette = 'pastel')

In [ ]:
# educational qualification breakdown
sns.countplot(data = fact_submissions,
              y = 'educational_qualification_en',
              palette = 'Accent')

In [ ]:
# race breakdown
sns.countplot(data = fact_submissions,
              y = 'race_en',
              palette = 'Set2')

In [ ]:
race_gender_breakdown = fact_submissions.groupby(['race_en', 'gender_en']).size().to_frame('count')
race_gender_breakdown.reset_index(inplace = True)

In [ ]:
race_gender_breakdown.info()

In [ ]:
race_gender_breakdown = race_gender_breakdown.pivot(index = 'gender_en', columns = 'race_en', values = 'count')

In [ ]:
race_gender_breakdown

In [ ]:
# race breakdown
sns.countplot(data = fact_submissions,
              y = 'race_en',
              hue = 'gender_en',
              palette = 'Set2')

In [ ]:
# edu race breakdown
edu_race_breakdown = fact_submissions.groupby(['race_en', 'educational_qualification_en']).size().to_frame('count')
edu_race_breakdown.reset_index(inplace = True)
edu_race_breakdown = edu_race_breakdown.pivot(index = 'educational_qualification_en', columns = 'race_en', values = 'count')
edu_race_breakdown

In [ ]:
# edu gender breakdown
edu_race_breakdown = fact_submissions.groupby(['gender_en', 'educational_qualification_en']).size().to_frame('count')
edu_race_breakdown.reset_index(inplace = True)
edu_race_breakdown = edu_race_breakdown.pivot(index = 'educational_qualification_en', columns = 'gender_en', values = 'count')
edu_race_breakdown